In [88]:
# Packages used

using DataFrames,
    CSV, 
    Statistics, 
    FinancialToolbox,
    Dates,
    Cascadia,
    Gumbo,
    HTTP

In [472]:
# Importing CSVs

amzn_data = CSV.read("amzn_itm_calls.csv", DataFrame)
amzn_itm_calls = CSV.read("amzn_data.csv", DataFrame)

,Column1,contractSymbol,lastTradeDate,strike,lastPrice,impliedVolatility
,Int64,String31,String31,Float64,Float64,Float64
1,0,AMZN240119C01600000,2021-12-29 19:33:53,1600.0,1874.0,0.455414
2,1,AMZN240119C01620000,2021-12-13 20:20:40,1620.0,1878.0,0.452032
3,2,AMZN240119C01640000,2021-11-10 11:48:11,1640.0,1692.02,0.567616
4,3,AMZN240119C01660000,2021-12-22 18:41:23,1660.0,1811.5,0.445249
5,4,AMZN240119C01680000,2021-12-30 14:40:17,1680.0,1793.0,0.441847
6,5,AMZN240119C01700000,2021-12-29 15:12:54,1700.0,1772.0,0.438579
7,6,AMZN240119C01720000,2021-11-30 16:22:49,1720.0,1891.0,0.47756
8,7,AMZN240119C01740000,2021-12-17 19:50:52,1740.0,1753.0,0.43228
9,8,AMZN240119C01760000,2021-11-17 18:51:22,1760.0,1899.9,0.500442


In [316]:
# Volatility Code

three_yrs = 252*3 
σ = std(last(amzn_data[!,"Returns"], three_yrs))*sqrt(252)

#  Dividend Code
d = 0

# Current price code

amzn_ex_price = last(amzn_data[!,"Adj Close"])
σ

0.29378519174120554

In [317]:
# risk_free_rate scraping

url = "https://www.bloomberg.com/markets/rates-bonds/government-bonds/us"
r = HTTP.get(url)
h = parsehtml(String(r.body))
body = h.root[2]
risk_free_raw_1 = eachmatch(Selector("#content > div > div >
        div.section-front__main-content > div.data-tables.first > div >
        table > tbody > tr:nth-child(6) > td:nth-child(4) > span"), body)

risk_free_raw_2 = (text(risk_free_raw[1]))

"1.51%"

In [318]:
# Risk free float conversion

risk_free_rate = replace(risk_free_raw_2, "%" => "" )
risk_free_float = parse(Float64, risk_free_rate)/100

0.0151

In [319]:
# Time delta code

today = Dates.today()
due_date = Dates.Date("2024-01-19")
T = (Dates.value(due_date - today))/252

2.9682539682539684

In [320]:
function bstest(row)
    strikep = amzn_itm_calls[!,"strike"][row]
    blsp = blsprice(amzn_ex_price, strikep, risk_free_float, T, σ, d )
    actualp = amzn_itm_calls[!,"lastPrice"][row]
    println("Black-Scholes Price:", blsp)
    print("Actual Price:", actualp)
end

bstest (generic function with 1 method)

In [471]:
rownumber = size(amzn_itm_calls, 1)
randrow = rand(1:rownumber)
bstest(randrow)

Black-Scholes Price:752.1993441753975
Actual Price:693.68